In [37]:
import pandas as pd
import os
import sys
import logging
import pandas as pd
from google.cloud import bigquery
from hashlib import md5
from typing import List
import shortuuid
import numpy
import pyarrow

In [ ]:
# df = pd.read_csv("./spreadspoke_scores.csv")

# df = df[df["schedule_season"] > 1978]

# df = df.drop(columns="weather_detail")

# display(df[df["schedule_playoff"] == True])

In [134]:
DATA_DIR = "./spreadspoke_scores.csv"

PROJECT_NAME = "team-week-10"
DATASET_NAME = "nfl_sports_betting"

TABLE_METADATA = {
    "point_spread": {
        "table_name": "fct_points_spread",
        "schema": [
            bigquery.SchemaField("uuid", "string", mode="REQUIRED"),
            bigquery.SchemaField("team_favorite_id", "string", mode="REQUIRED"),
            bigquery.SchemaField("team_home", "string", mode="REQUIRED"),
            bigquery.SchemaField("team_away", "string", mode="Required"),
            bigquery.SchemaField("spread_favorite", "float64", mode="Required"),
        ]
    },
    "over_under": {
        "table_name": "fct_over_under",
        "schema": [
            bigquery.SchemaField("uuid", "string", mode="REQUIRED"),
            bigquery.SchemaField("over_under_line", "float64", mode="NULLABLE"),
            bigquery.SchemaField("team_favorite_id", "string", mode="REQUIRED"),
            bigquery.SchemaField("team_home", "string", mode="REQUIRED"),
            bigquery.SchemaField("score_home", "int64", mode="REQUIRED"),
            bigquery.SchemaField("score_away", "int64", mode="REQUIRED"),
            bigquery.SchemaField("team_away", "string", mode="REQUIRED"),
            bigquery.SchemaField("total_score", "int64", mode="REQUIRED"),
            bigquery.SchemaField("over", "boolean", mode="REQUIRED"),
        ]
    },
    "games": {
        "table_name": "dim_games",
        "schema": [
            bigquery.SchemaField("date", "date", mode="REQUIRED"),
            bigquery.SchemaField("season", "int64", mode="REQUIRED"),
            bigquery.SchemaField("week", "string", mode="REQUIRED"),
            bigquery.SchemaField("playoff", "boolean", mode="REQUIRED"),
            bigquery.SchemaField("team_home", "string", mode="REQUIRED"),
            bigquery.SchemaField("score_home", "int64", mode="REQUIRED"),
            bigquery.SchemaField("score_away", "int64", mode="REQUIRED"),
            bigquery.SchemaField("team_away", "string", mode="REQUIRED"),
            bigquery.SchemaField("stadium", "string", mode="REQUIRED"),
            bigquery.SchemaField("temperature_F", "float64", mode="NULLABLE"),
            bigquery.SchemaField("wind_mph", "float64", mode="NULLABLE"),
            bigquery.SchemaField("humidity_%", "float64", mode="NULLABLE"),
            bigquery.SchemaField("uuid", "string", mode="REQUIRED")
        ]
    },
    "lookup_teams": {
        "table_name": "lookup_teams",
        "schema": [
            bigquery.SchemaField("team_id", "string", mode="REQUIRED"),
            bigquery.SchemaField("team_name", "string", mode="REQUIRED"),
            bigquery.SchemaField("conference_string", "string", mode="REQUIRED"),
            bigquery.SchemaField("division", "string", mode="REQUIRED"),
        ]
    }
}

logging.basicConfig(
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)

In [ ]:
dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
client = bigquery.Client()

def create_dataset(client: bigquery.Client, dataset_id: str, location: str = "US"):
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = location
    dataset = client.create_dataset(dataset, exists_ok=True)
    logger.info(f"Created dataset: {dataset.full_dataset_id}")

create_dataset(client, dataset_id)

In [105]:
df = pd.read_csv("./spreadspoke_scores.csv", header=0)
display(df.head(2))

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN


In [117]:
import uuid
import numpy as np
df = pd.read_csv("./spreadspoke_scores.csv", header=0)

df = df[df["schedule_season"] > 1978]

df['uuid'] = [uuid.uuid4() for _ in range(len(df.index))]
df = df.astype({"uuid": "string"})

# df = df.astype({"score_home": int, "score_away": int, "weather_temperature": int, "weather_wind_mph": int, "weather_humidity": int}, errors="ignore")

df.dropna(subset=["score_home", "score_away"], inplace=True)

df = df.reset_index(drop=True)


df["over_under_line"] = pd.to_numeric(df["over_under_line"], errors="coerce")

# display(df)

games_df = df.drop(columns=["weather_detail", "stadium_neutral", "over_under_line", "spread_favorite", "team_favorite_id"])

# games_df.dropna(subset=["score_home", "score_away"], inplace=True)


games_df = games_df.rename(columns={"weather_humidity": "humidity_%", "weather_wind_mph": "wind_mph", "weather_temperature": "temperature_F", "schedule_date": "date", "schedule_season": "season", "schedule_week": "week", "schedule_playoff": "playoff"})

games_df["date"] = pd.to_datetime(games_df["date"])

games_df = games_df.astype({"score_home": int, "score_away": int, "temperature_F": int, "wind_mph": int, "humidity_%": int}, errors="ignore")

display(games_df.head(2))
games_df.info()



,date,season,week,playoff,team_home,score_home,score_away,team_away,stadium,temperature_F,wind_mph,humidity_%,uuid
0,1979-09-01,1979,1,False,Tampa Bay Buccaneers,31,16,Detroit Lions,Houlihan's Stadium,79.0,9.0,87.0,943e8288-cb34-4e6f-a091-9775975b6eb7
1,1979-09-02,1979,1,False,Buffalo Bills,7,9,Miami Dolphins,Ralph Wilson Stadium,74.0,15.0,74.0,af3854c6-7a8b-43a7-aacf-0d905bc652ac


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11255 entries, 0 to 11254
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           11255 non-null  datetime64[ns]
 1   season         11255 non-null  int64         
 2   week           11255 non-null  object        
 3   playoff        11255 non-null  bool          
 4   team_home      11255 non-null  object        
 5   score_home     11255 non-null  int64         
 6   score_away     11255 non-null  int64         
 7   team_away      11255 non-null  object        
 8   stadium        11255 non-null  object        
 9   temperature_F  10130 non-null  float64       
 10  wind_mph       10114 non-null  float64       
 11  humidity_%     6358 non-null   float64       
 12  uuid           11255 non-null  string        
dtypes: bool(1), datetime64[ns](1), float64(3), int64(3), object(4), string(1)
memory usage: 1.0+ MB


In [8]:
games_table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['games']['table_name']}"
games_schema = schema=TABLE_METADATA['games']['schema']

def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """

    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"


    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result() 

    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

load_table(games_df, client, games_table_name, games_schema)

[INFO ][2024-01-10 13:20:45,954][898524737:0032] : loading table: 'team-week-10.nfl_sports_betting.dim_games'
[DEBUG][2024-01-10 13:20:46,981][connectionpool:0546] : https://bigquery.googleapis.com:443 "POST /upload/bigquery/v2/projects/team-week-10/jobs?uploadType=multipart HTTP/1.1" 200 2800
[DEBUG][2024-01-10 13:20:47,088][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/team-week-10/jobs/4a154ceb-916f-4437-a930-b6d04527e582?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2024-01-10 13:20:47,089][retry:0233] : Retrying due to , sleeping 0.7s ...
[DEBUG][2024-01-10 13:20:47,906][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/team-week-10/jobs/4a154ceb-916f-4437-a930-b6d04527e582?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2024-01-10 13:20:47,907][retry:0233] : Retrying due to , sleeping 0.2s ...
[DEBUG][2024-01-10 13:20:48,210][connectionpool:0546] : https://bigquery.googleapis.com:443 "

In [118]:
from google.cloud import bigquery

client = bigquery.Client()
print(f"BigQuery Project: {client.project}")

# list datasets
print("Listing datasets:")

for dataset in client.list_datasets():
    dataset_id = dataset.dataset_id
    print(f"dataset id: `{dataset_id}`, full_name: `{dataset.full_dataset_id}`, labels (tags): {dataset.labels}")

[DEBUG][2024-01-10 17:44:59,279][_default:0255] : Checking /home/jarvismwynn/.creds/jarvis.json for explicit credentials as part of auth process...
[DEBUG][2024-01-10 17:44:59,281][_default:0255] : Checking /home/jarvismwynn/.creds/jarvis.json for explicit credentials as part of auth process...
BigQuery Project: team-week-10
Listing datasets:
[DEBUG][2024-01-10 17:44:59,283][retry:0282] : Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
[DEBUG][2024-01-10 17:44:59,305][requests:0185] : Making request: POST https://oauth2.googleapis.com/token
[DEBUG][2024-01-10 17:44:59,307][connectionpool:1052] : Starting new HTTPS connection (1): oauth2.googleapis.com:443
[DEBUG][2024-01-10 17:44:59,402][connectionpool:0546] : https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
[DEBUG][2024-01-10 17:44:59,404][connectionpool:1052] : Starting new HTTPS connection (1): bigquery.googleapis.com:443
[DEBUG][2024-01-10 17:44:59,672][connectionpo

In [121]:

over_under_df = df[["uuid", "over_under_line", "team_favorite_id", "team_home", "score_home", "score_away", "team_away"]]
# display(over_under_df.head(3))

def total_score(df):
    for _ in range(len(df.index)):
        df["total_score"] = df["score_home"] + df["score_away"]
    return df
total_score(over_under_df)




/tmp/ipykernel_157788/4178964628.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["total_score"] = df["score_home"] + df["score_away"]


,uuid,over_under_line,team_favorite_id,team_home,score_home,score_away,team_away,total_score
0,943e8288-cb34-4e6f-a091-9775975b6eb7,30.0,TB,Tampa Bay Buccaneers,31.0,16.0,Detroit Lions,47.0
1,af3854c6-7a8b-43a7-aacf-0d905bc652ac,39.0,MIA,Buffalo Bills,7.0,9.0,Miami Dolphins,16.0
2,ac8e2a49-f1b8-44fa-b2fb-89cd4c1c0a78,31.0,CHI,Chicago Bears,6.0,3.0,Green Bay Packers,9.0
3,88e61cea-79e4-4545-bc4f-fe4b83016640,31.5,DEN,Denver Broncos,10.0,0.0,Cincinnati Bengals,10.0
4,1311c24f-27e8-419c-ad2e-af7fcee96317,37.0,KC,Kansas City Chiefs,14.0,0.0,Baltimore Colts,14.0
...,...,...,...,...,...,...,...,...
11250,3ca8c2d3-158d-49b7-a2bd-ad9b428c0ac2,43.0,JAX,Tampa Bay Buccaneers,30.0,12.0,Jacksonville Jaguars,42.0
11251,bb9f6d89-e244-46b5-8d92-5850a1129ad0,42.0,SEA,Tennessee Titans,17.0,20.0,Seattle Seahawks,37.0
11252,af8478ae-0dcb-46af-9353-6c88e4de631d,40.5,KC,Kansas City Chiefs,14.0,20.0,Las Vegas Raiders,34.0
11253,b11328f8-a340-414a-b13e-7830bcbadad4,41.5,PHI,Philadelphia Eagles,33.0,25.0,New York Giants,58.0


In [120]:
over_under_df = over_under_df.astype({"score_home": int, "score_away": int, "over_under_line": float, "total_score": int}, errors="ignore")

over_under_df["over_under_line"] = pd.to_numeric(over_under_df["over_under_line"], errors="coerce")

for _ in range(len(over_under_df.index)):
    over_under_df["over"] = over_under_df["total_score"] > over_under_df["over_under_line"]
        # over_under_df["total_score"] == over_under_df["over_under_line"]
        
over_under_df = over_under_df.reset_index(drop=True)

display(over_under_df)
over_under_df.info()

,uuid,over_under_line,team_favorite_id,team_home,score_home,score_away,team_away,total_score,over
0,943e8288-cb34-4e6f-a091-9775975b6eb7,30.0,TB,Tampa Bay Buccaneers,31,16,Detroit Lions,47,True
1,af3854c6-7a8b-43a7-aacf-0d905bc652ac,39.0,MIA,Buffalo Bills,7,9,Miami Dolphins,16,False
2,ac8e2a49-f1b8-44fa-b2fb-89cd4c1c0a78,31.0,CHI,Chicago Bears,6,3,Green Bay Packers,9,False
3,88e61cea-79e4-4545-bc4f-fe4b83016640,31.5,DEN,Denver Broncos,10,0,Cincinnati Bengals,10,False
4,1311c24f-27e8-419c-ad2e-af7fcee96317,37.0,KC,Kansas City Chiefs,14,0,Baltimore Colts,14,False
...,...,...,...,...,...,...,...,...,...
11250,3ca8c2d3-158d-49b7-a2bd-ad9b428c0ac2,43.0,JAX,Tampa Bay Buccaneers,30,12,Jacksonville Jaguars,42,False
11251,bb9f6d89-e244-46b5-8d92-5850a1129ad0,42.0,SEA,Tennessee Titans,17,20,Seattle Seahawks,37,False
11252,af8478ae-0dcb-46af-9353-6c88e4de631d,40.5,KC,Kansas City Chiefs,14,20,Las Vegas Raiders,34,False
11253,b11328f8-a340-414a-b13e-7830bcbadad4,41.5,PHI,Philadelphia Eagles,33,25,New York Giants,58,True


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11255 entries, 0 to 11254
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uuid              11255 non-null  string 
 1   over_under_line   11193 non-null  float64
 2   team_favorite_id  11255 non-null  object 
 3   team_home         11255 non-null  object 
 4   score_home        11255 non-null  int64  
 5   score_away        11255 non-null  int64  
 6   team_away         11255 non-null  object 
 7   total_score       11255 non-null  int64  
 8   over              11255 non-null  bool   
dtypes: bool(1), float64(1), int64(3), object(3), string(1)
memory usage: 714.6+ KB


In [100]:
over_under_table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['over_under']['table_name']}"
over_under_schema = schema=TABLE_METADATA['over_under']['schema']

load_table(over_under_df, client, over_under_table_name, over_under_schema)

[INFO ][2024-01-10 16:38:07,577][898524737:0032] : loading table: 'team-week-10.nfl_sports_betting.fct_over_under'
[DEBUG][2024-01-10 16:38:08,589][connectionpool:0546] : https://bigquery.googleapis.com:443 "POST /upload/bigquery/v2/projects/team-week-10/jobs?uploadType=multipart HTTP/1.1" 200 2356
[DEBUG][2024-01-10 16:38:08,707][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/team-week-10/jobs/6310aab4-26ab-48b6-b094-33a400fbfe4c?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2024-01-10 16:38:08,708][retry:0233] : Retrying due to , sleeping 0.6s ...
[DEBUG][2024-01-10 16:38:09,448][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/team-week-10/jobs/6310aab4-26ab-48b6-b094-33a400fbfe4c?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2024-01-10 16:38:09,449][retry:0233] : Retrying due to , sleeping 0.8s ...
[DEBUG][2024-01-10 16:38:10,422][connectionpool:0546] : https://bigquery.googleapis.com:

In [129]:
point_spread_df = df[["uuid", "team_home", "score_home", "score_away", "team_away", "team_favorite_id", "spread_favorite"]]

point_spread_df = point_spread_df.astype({"score_home": int, "score_away": int})
display(point_spread_df.head(2))
point_spread_df.info()

,uuid,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite
0,943e8288-cb34-4e6f-a091-9775975b6eb7,Tampa Bay Buccaneers,31,16,Detroit Lions,TB,-3.0
1,af3854c6-7a8b-43a7-aacf-0d905bc652ac,Buffalo Bills,7,9,Miami Dolphins,MIA,-5.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11255 entries, 0 to 11254
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uuid              11255 non-null  string 
 1   team_home         11255 non-null  object 
 2   score_home        11255 non-null  int64  
 3   score_away        11255 non-null  int64  
 4   team_away         11255 non-null  object 
 5   team_favorite_id  11255 non-null  object 
 6   spread_favorite   11255 non-null  float64
dtypes: float64(1), int64(2), object(3), string(1)
memory usage: 615.6+ KB


In [135]:
point_spread_table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['point_spread']['table_name']}"
point_spread_schema = schema=TABLE_METADATA['point_spread']['schema']

load_table(point_spread_df, client, point_spread_table_name, point_spread_schema)

[INFO ][2024-01-10 17:54:47,504][898524737:0032] : loading table: 'team-week-10.nfl_sports_betting.fct_points_spread'
[DEBUG][2024-01-10 17:54:48,762][connectionpool:0546] : https://bigquery.googleapis.com:443 "POST /upload/bigquery/v2/projects/team-week-10/jobs?uploadType=multipart HTTP/1.1" 200 2122
[DEBUG][2024-01-10 17:54:48,917][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/team-week-10/jobs/b8f569bc-e5ea-4212-b358-660ae815b43e?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2024-01-10 17:54:48,919][retry:0233] : Retrying due to , sleeping 0.1s ...
[DEBUG][2024-01-10 17:54:49,171][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/team-week-10/jobs/b8f569bc-e5ea-4212-b358-660ae815b43e?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2024-01-10 17:54:49,173][retry:0233] : Retrying due to , sleeping 0.4s ...
[DEBUG][2024-01-10 17:54:49,691][connectionpool:0546] : https://bigquery.googleapis.c